# Python puro

# ¿Qué es Cpython?

No hay que confundir entre implementación y lenguaje. CPython es la implementación oficial del lenguaje de programación Python y está escrita en C.

Existen otras implementaciones como Jython, Pypy, IronPython, Brython, pypy.js, ..., que permiten obtener una serie de ventajas sobre la implementación oficial.

En el presente taller vamos a centrarnos en la implementación oficial, CPython, y en su alternativa natural cuando pensamos en correr código Python sin modificar y con mejoras de rendimiento, Pypy.

In [19]:
from math import sin, pi

def plate_displacement_py(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n):
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            for ii in range(max_i):
                for jj in range(max_j):
                    a_mn = 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
                    ww[ii][jj] += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                                   * sin(mm * pi * xx[ii][jj] / a)
                                   * sin(nn * pi * yy[ii][jj] / b)
                                   / (pi**4 * D))

Para preparar los valores de entrada vamos a usar numpy, por comodidad, pero al final todos los inputs serán objetos Python en lugar de numpy arrays.

In [20]:
# Plate geometry
a = 1.0  # m
b = 1.0  # m
h = 50e-3  # m

# Material properties
E = 69e9  # Pa
nu = 0.35

# Series terms
max_m = 16
max_n = 16

# Computation points
# NOTE: With an odd number of points the center of the place is included in
# the grid
NUM_POINTS = 101
max_i = NUM_POINTS
max_j = NUM_POINTS

# Load
P = 10e3  # N
xi = a / 2
eta = a / 2

# Flexural rigidity
D = h**3 * E / (12 * (1 - nu**2))

# ---

# Set up domain
x = np.linspace(0, a, num=NUM_POINTS)
y = np.linspace(0, b, num=NUM_POINTS)
xx, yy = np.meshgrid(x, y)

ww = np.zeros_like(xx)

xx = xx.tolist()
yy = yy.tolist()
ww = ww.tolist()

Veamos el resultado:

In [21]:
# Compute displacement field
plate_displacement_py(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n)

# Print maximum displacement
w_max = [[abs(i) for i in lst] for lst in ww[:]]
w_max = max([max(val) for val in w_max[:]])
print("Maximum displacement = %14.12f mm" % (w_max * 1e3))
print("alpha = %7.5f" % (w_max / (P * a**2 / D)))
print("alpha * P a^2 / D = %6.4f mm" % (0.01160 * P * a**2 / D * 1e3))

Maximum displacement = 0.141317840389 mm
alpha = 0.01158
alpha * P a^2 / D = 0.1416 mm


Y el tiempo de ejecución:

In [22]:
%timeit plate_displacement_py(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n)

1 loops, best of 3: 53.3 s per loop


# Versión en puro Pypy

## ¿Qué es pypy?

Pypy [pretende ser una implementación](http://pypy.org/features.html) que cumple 'totalmente' (o, al menos, lo máximo posible según crean los desarrolladores) con la especificación del lenguaje Python y que sea totalmente compatible con CPython.

Ahora mismo hay versiones para python 2.7 y 3.2.

<center><img src="./static/pypy_performance.png" width=600 height=400/></center>

In [23]:
%%pypy

from math import sin, pi
import time
import numpy as np

def plate_displacement(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n):
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            for ii in range(max_i):
                for jj in range(max_j):
                    a_mn = 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
                    ww[ii][jj] += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                                   * sin(mm * pi * xx[ii][jj] / a)
                                   * sin(nn * pi * yy[ii][jj] / b)
                                   / (pi**4 * D))

# Plate geometry
a = 1.0  # m
b = 1.0  # m
h = 50e-3  # m

# Material properties
E = 69e9  # Pa
nu = 0.35

# Series terms
max_m = 16
max_n = 16

# Computation points
# NOTE: With an odd number of points the center of the place is included in
# the grid
NUM_POINTS = 101
max_i = NUM_POINTS
max_j = NUM_POINTS

# Load
P = 10e3  # N
xi = a / 2
eta = a / 2

# Flexural rigidity
D = h**3 * E / (12 * (1 - nu**2))

# ---

# Set up domain
x = np.linspace(0, a, num=NUM_POINTS)
y = np.linspace(0, b, num=NUM_POINTS)
xx, yy = np.meshgrid(x, y)

ww = np.zeros_like(xx)

xx = xx.tolist()
yy = yy.tolist()
ww = ww.tolist()

# Compute displacement field
plate_displacement(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n)

# Print maximum displacement
w_max = [[abs(i) for i in lst] for lst in ww[:]]
w_max = max([max(val) for val in w_max[:]])
print("Maximum displacement = %14.12f mm" % (w_max * 1e3))
print("alpha = %7.5f" % (w_max / (P * a**2 / D)))
print("alpha * P a^2 / D = %6.4f mm" % (0.01160 * P * a**2 / D * 1e3))

t = []
times = 10
for i in range(times):
    t0 = time.time()
    plate_displacement(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n)
    t1 = time.time() - t0
    t.append(t1)
print(sum(t) / times)

Maximum displacement = 0.141317840389 mm
alpha = 0.01158
alpha * P a^2 / D = 0.1416 mm
2.76839942932
